In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score


%matplotlib inline

In [2]:
csv = 'reviews_clean.csv'
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

,text,stars
0,first admit not excite go la tavolta food snob...,4.0
1,walk around friday afternoon sit table bar wal...,1.0
2,not really find anything wrong place pastas pi...,5.0
3,great lunch today staff helpful assist selecti...,4.0
4,good selection class beers mains twice first t...,3.0


In [3]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864530 entries, 0 to 864529
Data columns (total 2 columns):
text     864530 non-null object
stars    864530 non-null float64
dtypes: float64(1), object(1)
memory usage: 13.2+ MB


In [4]:
x = my_df.text
y = my_df.stars

In [5]:
tvec = TfidfVectorizer(max_features=5000)
tvec.fit(my_df['text'])
x_tfidf = tvec.transform(my_df['text'])
# print(tvec.vocabulary_)

In [0]:
nb_model = MultinomialNB()
nb_model.fit(x_tfidf, y)

In [0]:
nb_predictions = nb_model.predict(x_tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(nb_predictions, y)*100)

In [0]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result

all_x_w2v = labelize_tweets_ug(x, 'all')

In [84]:
# model_ug_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
# model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 864530/864530 [00:00<00:00, 3276558.39it/s]


In [61]:
# for epoch in range(30):
#     model_ug_dbow.train(shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
#     model_ug_dbow.alpha -= 0.002
#     model_ug_dbow.min_alpha = model_ug_dbow.alpha

100%|██████████| 20000/20000 [00:00<00:00, 3444730.62it/s]


In [0]:
cores = multiprocessing.cpu_count()
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])

In [86]:
for epoch in range(30):
    model_ug_dmc.train(shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha

100%|██████████| 864530/864530 [00:00<00:00, 3473333.99it/s]


KeyboardInterrupt: 

In [68]:
model_ug_dmc.save('d2v_model_ug_dmc.doc2vec')
model_ug_dmc = Doc2Vec.load('d2v_model_ug_dmc.doc2vec')

In [69]:
model_ug_dmc.most_similar('good')


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('great', 0.988132119178772),
 ('like', 0.9839574694633484),
 ('\x00', 0.9836361408233643),
 ('go', 0.983223557472229),
 ('get', 0.9817840456962585),
 ('order', 0.9813681840896606),
 ('not', 0.9797091484069824),
 ('burger', 0.9794670939445496),
 ('place', 0.9784678220748901),
 ('come', 0.9780634641647339)]

In [73]:
model_ug_dmc.most_similar('truffle')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('garlic', 0.7460310459136963),
 ('seat', 0.7407455444335938),
 ('large', 0.7369706630706787),
 ('ysm', 0.7336738109588623),
 ('make', 0.7307016849517822),
 ('get', 0.7301536798477173),
 ('would', 0.7288651466369629),
 ('staff', 0.7278265953063965),
 ('well', 0.7276114225387573),
 ('order', 0.7263680696487427)]

In [75]:
model_ug_dmc.most_similar('rude')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('awesome', 0.8356685638427734),
 ('burger', 0.8354296684265137),
 ('nice', 0.8324081897735596),
 ('like', 0.8310285806655884),
 ('good', 0.8299099802970886),
 ('pizza', 0.8293734192848206),
 ('\x00', 0.828866720199585),
 ('food', 0.8259786367416382),
 ('one', 0.8227168321609497),
 ('know', 0.8219459056854248)]